In [4]:
# Import necessary packages
using SparseArrays
using StaticArrays
using LinearAlgebra
using IterativeSolvers
using AlgebraicMultigrid
using Printf

include("Grid.jl")
include("Stokes.jl")

unpack (generic function with 1 method)

In [9]:
mutable struct Marker
    x::MVector{2,Float64}# mutable but fixed-size vector of coordinates
    cell::MVector{2,Int16}# cell in which the marker is located
    rho::Float64
    function Marker()
       new(MVector{2,Float64}(-1.,-1.),MVector{2,Int16}(-1,-1),0.0) 
    end
end

mutable struct Markers
    # Currently, the markers only contains the array of markers and the number of markers in the array.
    m::Array{Marker,1}
    nmark::Int64
    function Markers(n::Integer)
       new( Array{Marker,1}(undef,n),0)
    end
    function Markers(grid::CartesianGrid; nmx::Integer=5,nmy::Integer=5)
       # distribute markers uniformly in the domain 
        # nmx and nmy are the number of markers per cell in the x and y directions, respectively.
        N = nmx*nmy*grid.nx*grid.ny # total number of markers
        mm = Array{Marker,1}(undef,N)
        mdx = grid.W/nmx/grid.nx
        mdy = grid.H/nmy/grid.ny
        k=1
        for i in 1:grid.ny-1
            for j in 1:grid.nx-1
                for ii in 1:nmy
                    for jj in 1:nmx
                        mm[k] = Marker()
                        mm[k].x[1] = mdx/2. + mdx*(jj-1) + mdx*nmx*(j-1)
                        mm[k].x[2] = mdy/2. + mdy*(ii-1) + mdy*nmy*(i-1) 
                        mm[k].cell[1] = j
                        mm[k].cell[2] = i
                        mm[k].rho = 0.
                        k+=1
                    end
                end
            end
        end
        new(mm,k-1)
    end
end

function find_cell(x::Float64,gridx::Vector{Float64},nx::Int ; guess::Integer=nothing)
    # find the cell in the array gridx that contains the current marker
    # first, see whether the initial guess is correct.
    lower::Int = 1
    upper::Int = nx
    if guess != nothing
        if x >= gridx[guess] && x <= gridx[guess+1]
            return guess
        else
            if x>=gridx[guess+1]
                lower = guess
            else
                upper = guess
            end
        end
     end
    # locate cell using bisection on lower,upper
    while lower-upper > 1 
        midpoint::Int = lower + floor((upper-lower)/2)
        if x >= gridx[midpoint]
            lower = midpoint
        else
            upper = midpoint
        end
    end
    return lower
end


initial_conditions_updown (generic function with 1 method)

In [7]:
# Set up the grid
nx = 51
ny = 101
W = 1e5
H = 1.5e5
gy = -10.0
gx =  0.0
markx = 10
marky = 10
grid = CartesianGrid(W,H,nx,ny)
markers = Markers(grid)


# rho,eta_s,eta_n = initial_conditions(grid)
# bc = BoundaryConditions(0,0,0,0)
# @time L,R = form_stokes(grid,eta_s,eta_n,bc,gx,gy)
# @time solution = L\R
# @time vx,vy,P = unpack(solution,grid)

Markers(Marker[Marker([196.07843137254903, 148.5148514851485], Int16[1, 1], 3300.0), Marker([588.2352941176471, 148.5148514851485], Int16[1, 1], 3300.0), Marker([980.3921568627452, 148.5148514851485], Int16[1, 1], 3300.0), Marker([1372.549019607843, 148.5148514851485], Int16[1, 1], 3300.0), Marker([1764.7058823529412, 148.5148514851485], Int16[1, 1], 3300.0), Marker([196.07843137254903, 445.54455445544556], Int16[1, 1], 3300.0), Marker([588.2352941176471, 445.54455445544556], Int16[1, 1], 3300.0), Marker([980.3921568627452, 445.54455445544556], Int16[1, 1], 3300.0), Marker([1372.549019607843, 445.54455445544556], Int16[1, 1], 3300.0), Marker([1764.7058823529412, 445.54455445544556], Int16[1, 1], 3300.0)  …  #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef], 125000)

1

In [9]:
a[1]= Marker()

Marker([-1.0, -1.0], Integer[-1, -1], 0.0)

In [10]:
a

1000-element Vector{Marker}:
    Marker([-1.0, -1.0], Integer[-1, -1], 0.0)
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
   ⋮
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef